In [ ]:
#Data Cleaning

import pandas as pd

# Step 1: Convert StartDate format + create numeric SubjectKey_N while retaining original SubjectKey
file_path = "Desktop/extracted_data_0402.csv"
df = pd.read_csv(file_path)

df["StartDate"] = pd.to_datetime(df["StartDate"], errors='coerce')
unique_subjects = df["SubjectKey"].dropna().unique()
subject_map = {old: str(i + 1) for i, old in enumerate(sorted(unique_subjects))}
df["SubjectKey_N"] = df["SubjectKey"].map(subject_map)

# Create patient_ID = SubjectKey_N + "_" + StartDate
df["patient_ID"] = df["SubjectKey_N"] + "_" + df["StartDate"].dt.strftime("%Y%m%d")

# Step 2: Filter BP values within 0–400
df["Value"] = pd.to_numeric(df["Value"], errors='coerce')
df = df[(df["Value"] >= 0) & (df["Value"] <= 400)]

# Step 3: Keep only patients aged 0–120
df = df[(df["SubjectAgeAtEvent"] >= 0) & (df["SubjectAgeAtEvent"] <= 120)]

# Step 4: Keep only systolic BP-related ItemOIDs
systolic_itemoids = [
    "I_VISIT_PASD", "I_VISIT_PASN",
    "I_VISIT_SBP_SITTING", "I_VISIT_SBP_SITTING2",
    "I_VISIT_PAS_SUPINO", "I_VISIT_PAS_SUP_2",
    "I_VISIT_PAS_PIES_1", "I_VISIT_PAS_PIES_2", "I_VISIT_PAS_PIES_3"
]
df = df[df["ItemOID"].isin(systolic_itemoids)]

# Step 5: Two-step priority filtering
# Step 5.1: Keep PASD and PASN without conditions
primary_df = df[df["ItemOID"].isin(["I_VISIT_PASD", "I_VISIT_PASN"])]

# Step 5.2: For the rest, keep based on priority by group
def filter_secondary_items(group):
    if group["ItemOID"].isin(["I_VISIT_SBP_SITTING", "I_VISIT_SBP_SITTING2"]).any():
        return group[group["ItemOID"].isin(["I_VISIT_SBP_SITTING", "I_VISIT_SBP_SITTING2"])]
    elif group["ItemOID"].isin(["I_VISIT_PAS_SUPINO", "I_VISIT_PAS_SUP_2"]).any():
        return group[group["ItemOID"].isin(["I_VISIT_PAS_SUPINO", "I_VISIT_PAS_SUP_2"])]
    else:
        return group[group["ItemOID"].isin(["I_VISIT_PAS_PIES_1", "I_VISIT_PAS_PIES_2", "I_VISIT_PAS_PIES_3"])]

remaining_df = df[~df["ItemOID"].isin(["I_VISIT_PASD", "I_VISIT_PASN"])]
filtered_secondary_df = remaining_df.groupby("patient_ID", group_keys=False).apply(filter_secondary_items)

# Combine final filtered results
df = pd.concat([primary_df, filtered_secondary_df], ignore_index=True)

# Step 6: Sort visits by SubjectKey and StartDate
df = df.sort_values(by=["SubjectKey", "StartDate"]).reset_index(drop=True)

# Step 7: Remove patients with only one visit and print stats
total_before_step7 = df.shape[0]
visit_counts = df.groupby("SubjectKey")["StartDate"].nunique()
valid_subjects = visit_counts[visit_counts > 1].index
removed_subjects = visit_counts[visit_counts <= 1].shape[0]
df = df[df["SubjectKey"].isin(valid_subjects)]
total_after_step7 = df.shape[0]
print(f"Step 7: Number of patients with only one visit removed: {removed_subjects}")
print(f"Step 7: Corresponding number of rows removed: {total_before_step7 - total_after_step7}")

# Step 8: Remove rows under patient_IDs that do not contain complete item pairs/triplets
def filter_complete_pairs(df, item_set):
    """
    For each patient_ID, check if all required items in item_set exist.
    If not all present, remove all such rows under that patient_ID.
    """
    pid_group = df[df["ItemOID"].isin(item_set)].groupby("patient_ID")["ItemOID"].apply(set)
    valid_pids = pid_group[pid_group.apply(lambda x: set(item_set).issubset(x))].index
    return df[~((df["ItemOID"].isin(item_set)) & (~df["patient_ID"].isin(valid_pids)))]

# Apply to sitting, supine, and standing sets
sitting_pair = ["I_VISIT_SBP_SITTING", "I_VISIT_SBP_SITTING2"]
supine_pair = ["I_VISIT_PAS_SUPINO", "I_VISIT_PAS_SUP_2"]
standing_triplet = ["I_VISIT_PAS_PIES_1", "I_VISIT_PAS_PIES_2", "I_VISIT_PAS_PIES_3"]

df = filter_complete_pairs(df, sitting_pair)
df = filter_complete_pairs(df, supine_pair)
df = filter_complete_pairs(df, standing_triplet)

print("Step 8: Removed rows lacking complete systolic BP item sets")

# Step 9: Save to desktop
output_path = "Desktop/cleaned_data_0607.csv"
df.to_csv(output_path, index=False)
print(f"Data saved to: {output_path}")

In [ ]:
# Create CSV1: extract rows with ItemOID = ["I_VISIT_PASD", "I_VISIT_PASN"]

import pandas as pd

# Read the cleaned dataset
file_path = "Desktop/cleaned_data_0607.csv"
df = pd.read_csv(file_path)

# Step CSV1: filter rows where ItemOID is PASD or PASN
csv1_df = df[df["ItemOID"].isin(["I_VISIT_PASD", "I_VISIT_PASN"])]

# Save the filtered result as CSV1
csv1_path = "Desktop/CSV1_PASD_or_PASN_only.csv"
csv1_df.to_csv(csv1_path, index=False)

print(f"CSV1 saved to: {csv1_path}")

In [ ]:
# Create CSV2: extract all rows except those with ItemOID = ["I_VISIT_PASD", "I_VISIT_PASN"]

import pandas as pd

# Read the cleaned dataset
file_path = "Desktop/cleaned_data_0607.csv"
df = pd.read_csv(file_path)

# Step CSV2: exclude rows where ItemOID is PASD or PASN, keep the rest
csv2_df = df[~df["ItemOID"].isin(["I_VISIT_PASD", "I_VISIT_PASN"])]

# Save the result as CSV2
csv2_path = "Desktop/CSV2_exclude_PASD_PASN.csv"
csv2_df.to_csv(csv2_path, index=False)

print(f"CSV2 saved to: {csv2_path}")

In [ ]:
# CSV1: Keep only one row per patient_ID; prioritize I_VISIT_PASN; if not available, keep I_VISIT_PASD

import pandas as pd

# Load CSV1 file
csv1_path = "Desktop/CSV1_PASD_or_PASN_only.csv"
df = pd.read_csv(csv1_path)

# Step 1: Prioritize I_VISIT_PASN
pasn_df = df[df["ItemOID"] == "I_VISIT_PASN"].drop_duplicates(subset="patient_ID", keep="first")

# Step 2: For patient_IDs without PASN, keep PASD
pasd_df = df[df["ItemOID"] == "I_VISIT_PASD"]
pasd_only_df = pasd_df[~pasd_df["patient_ID"].isin(pasn_df["patient_ID"])].drop_duplicates(subset="patient_ID", keep="first")

# Step 3: Combine both sets
final_df = pd.concat([pasn_df, pasd_only_df], ignore_index=True)

# Save final CSV1
output_path = "Desktop/CSV1_final_one_row_per_patientID.csv"
final_df.to_csv(output_path, index=False)

print(f"Final CSV1 file saved to: {output_path}")

In [ ]:
# CSV1: Blood pressure categorization based on SBP, sex, and time of measurement (day vs. night)

import pandas as pd

# Load the processed CSV1 file (one row per patient_ID, PASN prioritized)
csv1_path = "Desktop/CSV1_final_one_row_per_patientID.csv"
df = pd.read_csv(csv1_path)

# Define classification function
def classify_bp(row):
    gender = row["Sex"].strip().lower()
    sbp = row["Value"]
    source = row["ItemOID"]  # Identify PASN (night) or PASD (day)

    if pd.isna(sbp):
        return "Unclassified"

    # Night SBP rules (PASN)
    if source == "I_VISIT_PASN":
        if gender == "m":
            if sbp >= 120:
                return "High BP"
            elif sbp < 97:
                return "Low BP"
            else:
                return "Normal BP"
        elif gender == "f":
            if sbp >= 120:
                return "High BP"
            elif sbp < 92:
                return "Low BP"
            else:
                return "Normal BP"
    
    # Day SBP rules (PASD)
    elif source == "I_VISIT_PASD":
        if gender == "m":
            if sbp >= 135:
                return "High BP"
            elif sbp < 115:
                return "Low BP"
            else:
                return "Normal BP"
        elif gender == "f":
            if sbp >= 135:
                return "High BP"
            elif sbp < 105:
                return "Low BP"
            else:
                return "Normal BP"
    
    return "Unclassified"

# Apply classification
df["BP_Category"] = df.apply(classify_bp, axis=1)

# Save output
output_path = "Desktop/CSV1_with_BP_Category_Final.csv"
df.to_csv(output_path, index=False)

print(f"Blood pressure classification completed and saved to: {output_path}")

In [ ]:
# CSV2: Compute average SBP from valid composite measurements (sitting/supine/standing)
# One average row is added per patient_ID based on available full sets.

import pandas as pd

# Load the original CSV2 file
csv2_path = "Desktop/CSV2_exclude_PASD_PASN.csv"
df = pd.read_csv(csv2_path)

# Define three composite groups
groups = {
    "sitting_average": ["I_VISIT_SBP_SITTING", "I_VISIT_SBP_SITTING2"],
    "supine_average": ["I_VISIT_PAS_SUPINO", "I_VISIT_PAS_SUP_2"],
    "standing_average": ["I_VISIT_PAS_PIES_1", "I_VISIT_PAS_PIES_2", "I_VISIT_PAS_PIES_3"]
}

# Collect new average rows
new_rows = []

# Group by patient_ID
for pid, group in df.groupby("patient_ID"):
    for new_itemoid, itemoid_list in groups.items():
        subset = group[group["ItemOID"].isin(itemoid_list)]
        if set(itemoid_list).issubset(set(subset["ItemOID"])):  # Check if all required items are present
            avg_value = subset["Value"].mean()
            ref_row = subset.iloc[0].copy()
            ref_row["ItemOID"] = new_itemoid
            ref_row["Value"] = avg_value
            new_rows.append(ref_row)
            break  # Only keep one average group (in defined priority order)

# Combine original data with new average rows
if new_rows:
    new_rows_df = pd.DataFrame(new_rows)
    final_df = pd.concat([df, new_rows_df], ignore_index=True)
else:
    final_df = df.copy()

# Save final result
output_path = "Desktop/CSV2_with_averages.csv"
final_df.to_csv(output_path, index=False)

print(f"Average computation completed. Final file saved to: {output_path}")

In [ ]:
# Define sitting group
sitting_items = ["I_VISIT_SBP_SITTING", "I_VISIT_SBP_SITTING2"]

# Count number of records for each patient_ID in the sitting group
sitting_counts = df[df["ItemOID"].isin(sitting_items)].groupby("patient_ID").size()

# Identify patient_IDs with more than 2 records (indicating duplicates)
sitting_duplicates = sitting_counts[sitting_counts > 2]

# Print count and sample
print(f"There are {len(sitting_duplicates)} patient_IDs with duplicate records in the sitting group.")
print(sitting_duplicates.head())

# Check duplicates considering both ItemOID and patient_ID
repeated_items = df[df["ItemOID"].isin(sitting_items)]
repeated_detail = repeated_items.groupby(["patient_ID", "ItemOID"]).size().reset_index(name="count")
repeated_detail = repeated_detail[repeated_detail["count"] > 1]

# Print duplicate details
print("Duplicate record details:")
print(repeated_detail.head())

# Define supine group
supine_items = ["I_VISIT_PAS_SUPINO", "I_VISIT_PAS_SUP_2"]

# Count supine records per patient_ID
supine_counts = df[df["ItemOID"].isin(supine_items)].groupby("patient_ID").size()
supine_duplicates = supine_counts[supine_counts > 2]

# Print results
print(f"There are {len(supine_duplicates)} patient_IDs with duplicate records in the supine group.")
print(supine_duplicates.head())

# Define standing group
standing_items = ["I_VISIT_PAS_PIES_1", "I_VISIT_PAS_PIES_2", "I_VISIT_PAS_PIES_3"]

# Count standing records per patient_ID
standing_counts = df[df["ItemOID"].isin(standing_items)].groupby("patient_ID").size()
standing_duplicates = standing_counts[standing_counts > 3]

# Print results
print(f"There are {len(standing_duplicates)} patient_IDs with duplicate records in the standing group.")
print(standing_duplicates.head())

In [ ]:
# CSV2: Label SBP categories for composite average rows only (sitting/supine/standing)
# Classification is based on predefined SBP thresholds.

import pandas as pd

# Read the file that contains average rows
csv_path = "Desktop/CSV2_with_averages.csv"
df = pd.read_csv(csv_path)

# Step 1: Keep only the three types of average rows
average_df = df[df["ItemOID"].isin(["sitting_average", "supine_average", "standing_average"])].copy()

# Step 2: Add BP_Category column based on average value
def classify_secondary_bp(value):
    if pd.isna(value):
        return "Unclassified"
    elif value >= 140:
        return "High BP"
    elif value < 90:
        return "Low BP"
    else:
        return "Normal BP"

average_df["BP_Category"] = average_df["Value"].apply(classify_secondary_bp)

# Step 3: Save the new file
output_path = "Desktop/CSV2_average_with_BP_Category.csv"
average_df.to_csv(output_path, index=False)

print(f"Secondary BP labeling completed. File saved to: {output_path}")

In [ ]:
# Merge CSV1 and CSV2 to create final CSV3 with complete BP classification per patient

import pandas as pd

# Read cleaned CSV1 and CSV2 files
csv1 = pd.read_csv("Desktop/CSV1_with_BP_Category_Final.csv")
csv2 = pd.read_csv("Desktop/CSV2_average_with_BP_Category.csv")

# Drop unused columns if they exist
csv1 = csv1.drop(columns=["StudyEventRepeatKey"], errors="ignore")
csv2 = csv2.drop(columns=["StudyEventRepeatKey"], errors="ignore")

# Rename columns to prevent conflicts after merge
csv1 = csv1.rename(columns={
    "ItemOID": "ItemOID_CSV1",
    "Value": "Value_CSV1",
    "BP_Category": "BP_Category_CSV1",
    "SubjectKey": "SubjectKey_CSV1",
    "Sex": "Sex_CSV1",
    "SubjectAgeAtEvent": "SubjectAgeAtEvent_CSV1",
    "StartDate": "StartDate_CSV1",
    "SubjectKey_N": "SubjectKey_N_CSV1"
})

csv2 = csv2.rename(columns={
    "ItemOID": "ItemOID_CSV2",
    "Value": "Value_CSV2",
    "BP_Category": "BP_Category_CSV2",
    "SubjectKey": "SubjectKey_CSV2",
    "Sex": "Sex_CSV2",
    "SubjectAgeAtEvent": "SubjectAgeAtEvent_CSV2",
    "StartDate": "StartDate_CSV2",
    "SubjectKey_N": "SubjectKey_N_CSV2"
})

# Select relevant columns from both datasets
csv1_base = csv1[["patient_ID", "SubjectKey_CSV1", "Sex_CSV1", "SubjectAgeAtEvent_CSV1", "StartDate_CSV1", "SubjectKey_N_CSV1",
                  "ItemOID_CSV1", "Value_CSV1", "BP_Category_CSV1"]]
csv2_base = csv2[["patient_ID", "SubjectKey_CSV2", "Sex_CSV2", "SubjectAgeAtEvent_CSV2", "StartDate_CSV2", "SubjectKey_N_CSV2",
                  "ItemOID_CSV2", "Value_CSV2", "BP_Category_CSV2"]]

# Merge by patient_ID
merged = pd.merge(csv1_base, csv2_base, on="patient_ID", how="outer")

# Fill demographic fields using CSV1 first, fallback to CSV2
merged["SubjectKey"] = merged["SubjectKey_CSV1"].combine_first(merged["SubjectKey_CSV2"])
merged["Sex"] = merged["Sex_CSV1"].combine_first(merged["Sex_CSV2"])
merged["SubjectAgeAtEvent"] = merged["SubjectAgeAtEvent_CSV1"].combine_first(merged["SubjectAgeAtEvent_CSV2"])
merged["StartDate"] = merged["StartDate_CSV1"].combine_first(merged["StartDate_CSV2"])
merged["SubjectKey_N"] = merged["SubjectKey_N_CSV1"].combine_first(merged["SubjectKey_N_CSV2"])

# Keep final cleaned columns
final_df = merged[[
    "patient_ID", "SubjectKey", "Sex", "SubjectAgeAtEvent", "StartDate", "SubjectKey_N",
    "ItemOID_CSV1", "Value_CSV1", "BP_Category_CSV1",
    "ItemOID_CSV2", "Value_CSV2", "BP_Category_CSV2"
]]

# Save output
output_path = "Desktop/CSV3_merged_all_patients_filled.csv"
final_df.to_csv(output_path, index=False)

print(f"CSV3 merge and repair completed. File saved to: {output_path}")

In [ ]:
# Add BP_Classification based on BP_Category_CSV1 and BP_Category_CSV2 using a given Classification map

import pandas as pd

# Load the merged CSV3 file
csv3_path = "Desktop/CSV3_merged_all_patients_filled.csv"
df = pd.read_csv(csv3_path)

# Define classification mapping based on CSV1 and CSV2 categories
classification_map = {
    ("High BP", "High BP"): "High BP",
    ("High BP", "Normal BP"): "Masked Hypertension",
    ("High BP", "Low BP"): "Masked Hypertension",
    ("Normal BP", "High BP"): "High BP White coat",
    ("Normal BP", "Normal BP"): "Normal BP",
    ("Normal BP", "Low BP"): "Whitecoat Low blood pressure",
    ("Low BP", "High BP"): "High BP White coat",
    ("Low BP", "Normal BP"): "Masked Low blood pressure",
    ("Low BP", "Low BP"): "Low BP"
}

# Classification function based on availability of both categories
def classify(row):
    cat1 = row["BP_Category_CSV1"]
    cat2 = row["BP_Category_CSV2"]

    if pd.notna(cat1) and pd.isna(cat2):
        return cat1
    elif pd.notna(cat1) and pd.notna(cat2):
        return classification_map.get((cat1, cat2), "Unclassified")
    else:
        return "Unclassified"

# Apply the classification
df["BP_Classification"] = df.apply(classify, axis=1)

# Save final result
output_path = "Desktop/CSV3_with_BP_Classification_final.csv"
df.to_csv(output_path, index=False)

print(f"Final classification saved to: {output_path}")

In [ ]:
# CSV3: Generate trajectory and calculate transition age and time gap based on BP_Classification

import pandas as pd

# Load data
file_path = "CSV3_BP_Matching_Final_With_Classification.csv"
# This file is generated later from classification code after data matching and cleaning
# See: 'Label BP_Classification based on BP_Category_CSV1 and CSV2 (Final Classification)'
df = pd.read_csv(file_path)

# Ensure StartDate is in datetime format
df["StartDate"] = pd.to_datetime(df["StartDate"])

# Sort by SubjectKey and StartDate
df_sorted = df.sort_values(by=["SubjectKey", "StartDate"]).reset_index(drop=True)

# Initialize new columns
df_sorted["trajectory"] = ""
df_sorted["transition_age"] = None
df_sorted["time_dif_visit_Days"] = None

# Process each SubjectKey group
for subject, group in df_sorted.groupby("SubjectKey"):
    group = group.sort_values(by="StartDate").reset_index()
    for i in range(1, len(group)):
        idx = group.loc[i, "index"]
        current_bp = group.loc[i, "BP_Classification"]
        previous_bp = group.loc[i - 1, "BP_Classification"]
        date_1 = group.loc[i, "StartDate"]
        date_2 = group.loc[i - 1, "StartDate"]
        age_1 = group.loc[i, "SubjectAgeAtEvent"]

        df_sorted.at[idx, "trajectory"] = f"{previous_bp} - {current_bp}"
        df_sorted.at[idx, "transition_age"] = age_1
        df_sorted.at[idx, "time_dif_visit_Days"] = (date_1 - date_2).days

# Save result
output_path = "Desktop/CSV3_BP_Matching_With_Trajectory.csv"
df_sorted.to_csv(output_path, index=False)

print(f"transition_age uses the second visit's age. File saved to: {output_path}")

In [ ]:
# Plot age distribution for four specific BP classifications using boxplot and violin plot

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory.csv"
df = pd.read_csv(file_path)

# Filter for four target BP classifications
target_classes = [
    "Masked Hypertension",
    "High BP White coat",
    "Whitecoat Low BP",
    "Masked Low BP"
]
df_filtered = df[df["BP_Classification"].isin(target_classes)]

# Set seaborn style
sns.set(style="whitegrid")

# Create boxplot of transition_age by BP_Classification
plt.figure(figsize=(10, 6))
sns.boxplot(x="BP_Classification", y="SubjectAgeAtEvent", data=df_filtered, palette="Set2")
plt.title("Boxplot of Age by BP Classification")
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig("Desktop/boxplot_BP_Classification_age.png")
plt.show()

# Create violin plot of transition_age by BP_Classification
plt.figure(figsize=(10, 6))
sns.violinplot(x="BP_Classification", y="SubjectAgeAtEvent", data=df_filtered, inner="box", palette="Set2")
plt.title("Violin Plot of Age by BP Classification")
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig("Desktop/violinplot_BP_Classification_age.png")
plt.show()

In [ ]:
# Plot scatter plots of age vs. time_dif_visit_Days for each trajectory group

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory.csv"
df = pd.read_csv(file_path)

# Ensure time_dif_visit_Days is numeric
df["time_dif_visit_Days"] = pd.to_numeric(df["time_dif_visit_Days"], errors="coerce")

# Define trajectory types and associated colors
trajectory_list = [
    ("High BP - Normal BP", "blue"),
    ("High BP - Low BP", "orange"),
    ("High BP - High BP", "gold"),
    ("Normal BP - High BP", "purple"),
    ("Normal BP - Low BP", "green"),
    ("Normal BP - Normal BP", "gray"),
    ("Low BP - High BP", "brown"),
    ("Low BP - Normal BP", "red"),
    ("Low BP - Low BP", "teal")
]

# Generate scatter plots for each trajectory type
for trajectory_type, color in trajectory_list:
    filtered_df = df[df["trajectory"] == trajectory_type].dropna(subset=["time_dif_visit_Days", "age"])
    n = len(filtered_df)

    if n == 0:
        print(f"No data for trajectory: {trajectory_type}")
        continue

    plt.figure(figsize=(10, 6))
    plt.scatter(filtered_df["age"], filtered_df["time_dif_visit_Days"], color=color, alpha=0.6)

    max_y = filtered_df["time_dif_visit_Days"].max()
    yticks = np.arange(0, max_y + 500, 500)
    plt.yticks(yticks)

    plt.title(f"{trajectory_type} (n = {n})\nage vs. time_dif_visit_Days")
    plt.xlabel("Age")
    plt.ylabel("Time Difference Between Visits (Days)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Create a boxplot showing transition_age by 9 BP trajectory types using Seaborn Set2 palette

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("Desktop/CSV3_BP_Matching_With_Trajectory.csv")
df = df.dropna(subset=["trajectory", "transition_age"])

# Keep only the 9 defined BP trajectory types
valid_trajectories = [
    "High BP - Normal BP", "High BP - Low BP", "High BP - High BP",
    "Normal BP - High BP", "Normal BP - Low BP", "Normal BP - Normal BP",
    "Low BP - High BP", "Low BP - Normal BP", "Low BP - Low BP"
]
df = df[df["trajectory"].isin(valid_trajectories)]

# Count samples for each trajectory type
trajectory_counts = df["trajectory"].value_counts().to_dict()

# Create boxplot
plt.figure(figsize=(14, 6))
ax = sns.boxplot(x="trajectory", y="transition_age", data=df, palette="Set2")

# Add sample size (n=) labels
for i, label in enumerate(ax.get_xticklabels()):
    traj = label.get_text()
    count = trajectory_counts.get(traj, 0)
    ax.text(i, ax.get_ylim()[0] - 1, f"n={count}", ha='center', va='top', fontsize=10)

plt.xticks(rotation=45)
plt.title("Transition Age by BP Trajectory Type")
plt.xlabel("BP Trajectory")
plt.ylabel("Transition Age")
if ax.get_legend():
    ax.legend_.remove()

plt.tight_layout()
plt.show()

In [ ]:
# Plot violin + strip plots for transition_age across 9 BP trajectory types using Set2 palette

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("Desktop/CSV3_BP_Matching_With_Trajectory.csv")
df = df.dropna(subset=["trajectory", "transition_age"])

# Keep only 9 specific trajectory types
valid_trajectories = [
    "High BP - Normal BP", "High BP - Low BP", "High BP - High BP",
    "Normal BP - High BP", "Normal BP - Low BP", "Normal BP - Normal BP",
    "Low BP - High BP", "Low BP - Normal BP", "Low BP - Low BP"
]
df = df[df["trajectory"].isin(valid_trajectories)]

# Count samples per trajectory
trajectory_counts = df["trajectory"].value_counts().to_dict()

# Create violin and strip plots
plt.figure(figsize=(16, 7))
sns.violinplot(x="trajectory", y="transition_age", data=df, palette="Set2", inner=None, dodge=False)
sns.stripplot(x="trajectory", y="transition_age", data=df, color="black", alpha=0.3, size=2, jitter=True)

# Add sample size (n=) labels
ax = plt.gca()
for i, label in enumerate(ax.get_xticklabels()):
    traj = label.get_text()
    count = trajectory_counts.get(traj, 0)
    ax.text(i, ax.get_ylim()[0] - 1, f"n={count}", ha='center', va='top', fontsize=10)

plt.xticks(rotation=45)
plt.title("Transition Age by BP Trajectory Type (Violin + Strip, Set2 Palette)")
plt.xlabel("BP Trajectory")
plt.ylabel("Transition Age")
if ax.get_legend():
    ax.legend_.remove()

plt.tight_layout()
plt.show()

In [ ]:
# Plot 9 BP trajectory groups as individual scatter plots in a 3x3 grid layout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Load data
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory.csv"
df = pd.read_csv(file_path)

# Ensure numeric format
df["time_dif_visit_Days"] = pd.to_numeric(df["time_dif_visit_Days"], errors="coerce")

# Define trajectory types and colors
trajectory_list = [
    ("High BP - Normal BP", "blue"),
    ("High BP - Low BP", "orange"),
    ("High BP - High BP", "gold"),
    ("Normal BP - High BP", "purple"),
    ("Normal BP - Low BP", "green"),
    ("Normal BP - Normal BP", "gray"),
    ("Low BP - High BP", "brown"),
    ("Low BP - Normal BP", "red"),
    ("Low BP - Low BP", "teal")
]

# Determine global axis limits
all_transition_age = df["transition_age"].dropna()
all_time_diff = df["time_dif_visit_Days"].dropna()
x_min = all_transition_age.min() - 2
x_max = all_transition_age.max() + 2
y_min = 0
y_max = all_time_diff.max() + 500

# Create 3x3 subplot grid
fig = plt.figure(figsize=(18, 15))
gs = gridspec.GridSpec(3, 3, figure=fig)

for i, (trajectory_type, color) in enumerate(trajectory_list):
    row, col = divmod(i, 3)
    ax = fig.add_subplot(gs[row, col])
    
    filtered_df = df[df["trajectory"] == trajectory_type].dropna(subset=["time_dif_visit_Days", "transition_age"])
    n = len(filtered_df)
    
    if n > 0:
        ax.scatter(filtered_df["transition_age"], filtered_df["time_dif_visit_Days"],
                   color=color, alpha=0.6)
    
    ax.set_title(f"{trajectory_type} (n={n})", fontsize=12)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel("Transition Age")
    ax.set_ylabel("Time Difference (Days)")
    ax.grid(True)

plt.suptitle("Blood Pressure Trajectory Patterns (Original Classification: All 9 Groups)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# Generate pie charts of time difference distribution for each BP trajectory group

import pandas as pd
import matplotlib.pyplot as plt

# Load data
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory.csv"
df = pd.read_csv(file_path)

# Ensure numeric
df["time_dif_visit_Days"] = pd.to_numeric(df["time_dif_visit_Days"], errors="coerce")
df = df.dropna(subset=["trajectory", "time_dif_visit_Days"])

# Time difference bucket function
def categorize_gap(days):
    if days <= 30:
        return "≤30 days"
    elif days <= 90:
        return "31–90 days"
    elif days <= 180:
        return "91–180 days"
    elif days <= 365:
        return "181–365 days"
    elif days <= 730:
        return "1–2 years"
    elif days <= 1095:
        return "2–3 years"
    else:
        return "≥3 years"

# Define trajectory groups
trajectory_list = [
    "High BP - Normal BP",
    "High BP - Low BP",
    "High BP - High BP",
    "Normal BP - High BP",
    "Normal BP - Low BP",
    "Normal BP - Normal BP",
    "Low BP - High BP",
    "Low BP - Normal BP",
    "Low BP - Low BP"
]

# Generate pie chart for each trajectory
for trajectory in trajectory_list:
    subset = df[df["trajectory"] == trajectory]
    n = len(subset)
    if n == 0:
        continue

    # Bucket time difference
    subset["gap_category"] = subset["time_dif_visit_Days"].apply(categorize_gap)
    counts = subset["gap_category"].value_counts().sort_index()
    
    # Labels with count and percent
    labels = [f"{label} (n={count}, {count/n*100:.1f}%)" for label, count in counts.items()]
    
    # Plot
    plt.figure(figsize=(7, 6))
    plt.pie(counts, labels=labels, autopct="", startangle=90, wedgeprops=dict(width=0.5))
    plt.title(f"{trajectory}\nTime Difference Distribution (n={n})")
    plt.axis("equal")
    plt.tight_layout()
    plt.show()

In [ ]:
# *** Second Classification!
# Classification Strategy 2: Reclassify visit-based BP using matched ambulatory BP within 1 year (alternative to original classification)

import pandas as pd
from datetime import timedelta

# Load the merged file containing both visit (CSV2) and ambulatory (CSV1) BP records
df = pd.read_csv("Desktop/CSV3_merged_all_patients_filled.csv")

# Ensure StartDate is datetime
df["StartDate"] = pd.to_datetime(df["StartDate"], errors="coerce")

# Split into visit-based and ambulatory-based BP subsets
visit_df = df[df["ItemOID_CSV2"].notna()].copy()
ambulatory_df = df[df["ItemOID_CSV1"].notna()].copy()

# Initialize result list
result_rows = []

# Loop through each visit record (CSV2-based)
for idx, visit_row in visit_df.iterrows():
    patient_id = visit_row["patient_ID"]
    subject_key = visit_row["SubjectKey"]
    visit_date = visit_row["StartDate"]

    # Find all ambulatory BP rows for the same patient
    amb_subset = ambulatory_df[ambulatory_df["SubjectKey"] == subject_key]

    # Find ambulatory records within ±365 days of the visit
    amb_within_year = amb_subset[
        (amb_subset["StartDate"] - visit_date).abs() <= timedelta(days=365)
    ]

    # Copy the visit row
    combined_row = visit_row.copy()

    # Add each qualifying ambulatory BP classification as a new column
    for i, amb_row in amb_within_year.iterrows():
        amb_date = amb_row["StartDate"].strftime("%Y-%m-%d")
        new_col_name = f"BP_Category_CSV1_{amb_date}"
        combined_row[new_col_name] = amb_row["BP_Category_CSV1"]

    # Append combined row to result
    result_rows.append(combined_row)

# Combine all rows into a final DataFrame
final_result_df = pd.DataFrame(result_rows)

# Save to CSV
final_result_df.to_csv("Desktop/CSV3_BP_Matching_Result.csv", index=False)

print("Reclassification matching completed. Output saved to: Desktop/CSV3_BP_Matching_Result.csv")

In [ ]:
# Classification Strategy 2 (Nearest): For each visit BP record, match to the closest ambulatory BP record within 365 days

import pandas as pd
from datetime import timedelta

# Load the merged BP dataset
df = pd.read_csv("Desktop/CSV3_merged_all_patients_filled.csv")

# Ensure StartDate is in datetime format
df["StartDate"] = pd.to_datetime(df["StartDate"], errors="coerce")

# Split into visit-based and ambulatory-based subsets
visit_df = df[df["ItemOID_CSV2"].notna()].copy()
ambulatory_df = df[df["ItemOID_CSV1"].notna()].copy()

# Initialize columns to store nearest match
visit_df["BP_Category_CSV1_StartDate"] = pd.NaT
visit_df["BP_Category_CSV1_value"] = None

# Match each visit record to nearest ambulatory record within 1 year
for idx, row in visit_df.iterrows():
    subject = row["SubjectKey"]
    visit_date = row["StartDate"]

    # Filter ambulatory records for the same subject
    candidates = ambulatory_df[ambulatory_df["SubjectKey"] == subject].copy()
    candidates["DateDiff"] = (candidates["StartDate"] - visit_date).abs()

    # Keep only those within ±365 days
    candidates = candidates[candidates["DateDiff"] <= pd.Timedelta(days=365)]

    if not candidates.empty:
        # Select the record with the smallest time difference
        nearest = candidates.sort_values("DateDiff").iloc[0]
        visit_df.at[idx, "BP_Category_CSV1_StartDate"] = nearest["StartDate"]
        visit_df.at[idx, "BP_Category_CSV1_value"] = nearest["BP_Category_CSV1"]

# Save result
output_path = "Desktop/CSV3_BP_Matching_Nearest.csv"
visit_df.to_csv(output_path, index=False)
print(f"Nearest match completed. File saved to: {output_path}")

In [ ]:
# Classification Strategy 2 (Nearest): Match to the closest ambulatory BP within 365 days and record time difference

import pandas as pd
from datetime import timedelta

# Load the merged BP dataset
df = pd.read_csv("Desktop/CSV3_merged_all_patients_filled.csv")

# Ensure StartDate is in datetime format
df["StartDate"] = pd.to_datetime(df["StartDate"], errors="coerce")

# Split into visit-based and ambulatory-based subsets
visit_df = df[df["ItemOID_CSV2"].notna()].copy()
ambulatory_df = df[df["ItemOID_CSV1"].notna()].copy()

# Initialize new columns
visit_df["BP_Category_CSV1_StartDate"] = pd.NaT
visit_df["BP_Category_CSV1_value"] = None
visit_df["BP_Category_CSV1_time_difference_days"] = None

# Match each visit record to nearest ambulatory record and record time difference
for idx, row in visit_df.iterrows():
    subject = row["SubjectKey"]
    visit_date = row["StartDate"]

    # Filter ambulatory records for the same subject
    candidates = ambulatory_df[ambulatory_df["SubjectKey"] == subject].copy()
    candidates["DateDiff"] = (candidates["StartDate"] - visit_date).abs()

    # Keep only those within ±365 days
    candidates = candidates[candidates["DateDiff"] <= pd.Timedelta(days=365)]

    if not candidates.empty:
        nearest = candidates.sort_values("DateDiff").iloc[0]
        visit_df.at[idx, "BP_Category_CSV1_StartDate"] = nearest["StartDate"]
        visit_df.at[idx, "BP_Category_CSV1_value"] = nearest["BP_Category_CSV1"]
        visit_df.at[idx, "BP_Category_CSV1_time_difference_days"] = nearest["DateDiff"].days

# Save result
output_path = "Desktop/CSV3_BP_Matching_Nearest_WithTimeDiff.csv"
visit_df.to_csv(output_path, index=False)
print(f"Nearest match with time difference completed. File saved to: {output_path}")

In [ ]:
# Clean matched dataset: remove unmatched rows and subjects with only one record

import pandas as pd

# Load nearest matching result
df = pd.read_csv("Desktop/CSV3_BP_Matching_Nearest_WithTimeDiff.csv")

# Step 1: Remove rows with no matched CSV1 values
df = df[df["BP_Category_CSV1_value"].notna()]

# Step 2: Remove subjects with only one total record
subject_counts = df["SubjectKey"].value_counts()
df = df[df["SubjectKey"].isin(subject_counts[subject_counts > 1].index)]

# Save cleaned file
output_path = "Desktop/CSV3_BP_Matching_Nearest_Cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Cleaned file saved to: {output_path}")

In [ ]:
# Assign BP_Classification labels based on simplified CSV1 and CSV2 categories

import pandas as pd

# Load cleaned dataset
df = pd.read_csv("Desktop/CSV3_BP_Matching_Nearest_Cleaned.csv")

# Convert full category names to short codes: H (High), N (Normal), L (Low)
def simplify_category(val):
    if pd.isna(val):
        return None
    val = val.strip().lower()
    if "high" in val:
        return "H"
    elif "normal" in val:
        return "N"
    elif "low" in val:
        return "L"
    return None

df["BP_CSV1_Short"] = df["BP_Category_CSV1_value"].apply(simplify_category)
df["BP_CSV2_Short"] = df["BP_Category_CSV2"].apply(simplify_category)

# Define classification mapping based on short codes
classification_map = {
    ("H", "H"): "High BP",
    ("H", "N"): "Masked Hypertension",
    ("H", "L"): "Masked Hypertension",
    ("N", "H"): "High BP White coat",
    ("N", "N"): "Normal BP",
    ("N", "L"): "Whitecoat Low BP",
    ("L", "H"): "High BP White coat",
    ("L", "N"): "Masked Low BP",
    ("L", "L"): "Low BP"
}

# Generate final classification column
df["BP_Classification"] = df.apply(
    lambda row: classification_map.get((row["BP_CSV1_Short"], row["BP_CSV2_Short"]), "Unclassified"),
    axis=1
)

# Save final result
output_path = "Desktop/CSV3_BP_Matching_Final_With_Classification.csv"
df.to_csv(output_path, index=False)
print(f"File saved to: {output_path}")

In [ ]:
# Create a pie chart showing the proportion of patients with changed vs unchanged BP classification

import pandas as pd
import matplotlib.pyplot as plt

# Load the final classification file
file_path = "Desktop/CSV3_BP_Matching_Final_With_Classification.csv"
df = pd.read_csv(file_path)

# Ensure StartDate is in datetime format
df["StartDate"] = pd.to_datetime(df["StartDate"])

# Sort by SubjectKey and StartDate
df = df.sort_values(by=["SubjectKey", "StartDate"]).reset_index(drop=True)

# Remove patients with only one record
df = df[df.duplicated(subset="SubjectKey", keep=False)].reset_index(drop=True)

# Count unique BP_Classification types for each patient
bp_change_status = df.groupby("SubjectKey")["BP_Classification"].nunique().reset_index()

# Label as Changed if more than one type, else Unchanged
bp_change_status["BP_Change_Status"] = bp_change_status["BP_Classification"].apply(
    lambda x: "Changed" if x > 1 else "Unchanged"
)

# Plot pie chart
status_counts = bp_change_status["BP_Change_Status"].value_counts()
labels = status_counts.index
colors = ["#66c2a5", "#fc8d62"]
explode = [0.05 if label == "Changed" else 0 for label in labels]

plt.figure(figsize=(6, 6))
plt.pie(status_counts, labels=labels, autopct="%1.1f%%", colors=colors, explode=explode, startangle=90)
plt.title("Proportion of BP Changed vs Unchanged Patients")
plt.axis("equal")
plt.tight_layout()

# Save figure
plt.savefig("Desktop/bp_change_pie_chart.png")
plt.show()

In [ ]:
# Generate trajectory labels based on BP_Classification_Final sorted by patient and visit time

import pandas as pd

# Load the input file
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory.csv"
df = pd.read_csv(file_path)

# Step 1: Copy BP_Category_CSV1_value to a new column
df["BP_Classification_Final"] = df["BP_Category_CSV1_value"]

# Ensure StartDate is datetime
df["StartDate"] = pd.to_datetime(df["StartDate"])

# Sort by patient and date
df_sorted = df.sort_values(by=["SubjectKey", "StartDate"]).reset_index(drop=True)

# Initialize new trajectory column
df_sorted["Trajectory_Final"] = ""

# Assign trajectory between consecutive visits
for subject, group in df_sorted.groupby("SubjectKey"):
    group = group.sort_values(by="StartDate").reset_index()
    for i in range(1, len(group)):
        idx = group.loc[i, "index"]
        current_bp = group.loc[i, "BP_Classification_Final"]
        previous_bp = group.loc[i - 1, "BP_Classification_Final"]
        trajectory = f"{previous_bp} - {current_bp}"
        df_sorted.at[idx, "Trajectory_Final"] = trajectory

# Save output
output_path = "Desktop/CSV3_BP_Matching_With_Trajectory_Final.csv"
df_sorted.to_csv(output_path, index=False)

print(f"File saved to: {output_path}")

In [ ]:
# Plot transition age vs. time difference (days) for each of the 9 BP trajectory types using Final Classification

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the final trajectory-labeled file
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory_Final.csv"
df = pd.read_csv(file_path)

# Ensure numeric format
df["time_dif_visit_Days"] = pd.to_numeric(df["time_dif_visit_Days"], errors="coerce")

# Define 9 trajectory types and corresponding colors
trajectory_list = [
    ("High BP - Normal BP", "blue"),
    ("High BP - Low BP", "orange"),
    ("High BP - High BP", "gold"),
    ("Normal BP - High BP", "purple"),
    ("Normal BP - Low BP", "green"),
    ("Normal BP - Normal BP", "gray"),
    ("Low BP - High BP", "brown"),
    ("Low BP - Normal BP", "red"),
    ("Low BP - Low BP", "teal")
]

# Loop through each trajectory type and plot scatter plot
for trajectory_type, color in trajectory_list:
    filtered_df = df[df["Trajectory_Final"] == trajectory_type].dropna(subset=["time_dif_visit_Days", "transition_age"])
    n = len(filtered_df)

    if n == 0:
        continue

    plt.figure(figsize=(10, 6))
    plt.scatter(filtered_df["transition_age"], filtered_df["time_dif_visit_Days"],
                color=color, alpha=0.6)

    max_y = filtered_df["time_dif_visit_Days"].max()
    yticks = np.arange(0, max_y + 500, 500)
    plt.yticks(yticks)

    plt.title(f"{trajectory_type} (n = {n})\nTransition Age vs. Time Difference Between Visits")
    plt.xlabel("Transition Age")
    plt.ylabel("Time Difference Between Visits (Days)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Create a pie chart based on BP_Classification proportions using final classification file

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the file
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory_Final.csv"
df = pd.read_csv(file_path)

# Count BP_Classification types
bp_counts = df["BP_Classification"].value_counts()
sizes = bp_counts.values
labels_raw = bp_counts.index
total = bp_counts.sum()

# Labels with count and percentage
labels_full = [f"{label} (n={count}, {count/total*100:.1f}%)" for label, count in zip(labels_raw, sizes)]
legend_labels = [f"{label} (n={count})" for label, count in zip(labels_raw, sizes)]

# Draw pie chart with annotations
fig, ax = plt.subplots(figsize=(10, 8))
wedges, texts = ax.pie(
    sizes,
    startangle=140,
    labeldistance=1.2,
    wedgeprops=dict(width=0.5),
    textprops=dict(size=12)
)

# Annotate each slice with arrow and text
for i, wedge in enumerate(wedges):
    angle = (wedge.theta2 + wedge.theta1) / 2
    x = np.cos(np.deg2rad(angle))
    y = np.sin(np.deg2rad(angle))
    ha = 'left' if x >= 0 else 'right'
    offset_y = (i % 2) * 0.05 if x < 0 else 0
    
    ax.annotate(
        labels_full[i],
        xy=(x * 0.8, y * 0.8),
        xytext=(x * 1.3, y * 1.3 + offset_y),
        arrowprops=dict(arrowstyle='->', color='gray'),
        ha=ha, va='center', fontsize=12
    )

# Add legend
plt.legend(wedges, legend_labels, title="BP Classification", loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=2, fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Based on the second classification method (Trajectory_Final), plot 9 scatterplots by BP trajectory type

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Load final trajectory classification file
file_path = "Desktop/CSV3_BP_Matching_With_Trajectory_Final.csv"
df = pd.read_csv(file_path)

# Ensure correct numeric format
df["time_dif_visit_Days"] = pd.to_numeric(df["time_dif_visit_Days"], errors="coerce")

# Define 9 trajectory types and colors
trajectory_list = [
    ("High BP - Normal BP", "blue"),
    ("High BP - Low BP", "orange"),
    ("High BP - High BP", "gold"),
    ("Normal BP - High BP", "purple"),
    ("Normal BP - Low BP", "green"),
    ("Normal BP - Normal BP", "gray"),
    ("Low BP - High BP", "brown"),
    ("Low BP - Normal BP", "red"),
    ("Low BP - Low BP", "teal")
]

# Determine axis ranges based on full data
all_transition_age = df["transition_age"].dropna()
all_time_diff = df["time_dif_visit_Days"].dropna()
x_min = all_transition_age.min() - 2
x_max = all_transition_age.max() + 2
y_min = 0
y_max = all_time_diff.max() + 500

# Create 3x3 grid of subplots
fig = plt.figure(figsize=(18, 15))
gs = gridspec.GridSpec(3, 3, figure=fig)

for i, (trajectory_type, color) in enumerate(trajectory_list):
    row, col = divmod(i, 3)
    ax = fig.add_subplot(gs[row, col])
    
    filtered_df = df[df["Trajectory_Final"] == trajectory_type].dropna(subset=["time_dif_visit_Days", "transition_age"])
    n = len(filtered_df)
    
    if n > 0:
        ax.scatter(filtered_df["transition_age"], filtered_df["time_dif_visit_Days"], color=color, alpha=0.6)
    
    ax.set_title(f"{trajectory_type} (n={n})", fontsize=12)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel("Transition Age")
    ax.set_ylabel("Time Difference (Days)")
    ax.grid(True)

plt.suptitle("Blood Pressure Trajectory Patterns (Trajectory_Final)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()